In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [53]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [7]:
len(y_cookie)

552

In [56]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [7]:
y_sentence = torch.load('sentence_target_dict.pt')

In [9]:
len(y_sentence)

240

Load Samples

In [8]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [11]:
len(X_cookie_filenames)

552

In [12]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [13]:
len(X_sent_filenames)

240

In [14]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [15]:
num_samples

792

In [16]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [137]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha
674 Dementia


In [142]:
target_674 = X_sentence_retder.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [17]:
glove_path = './PretrainedWordEmb/'

In [18]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [48]:
len(vocab_emb)

2188

In [160]:
pos_dict = torch.load('pos_dict.pt')

In [138]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_minibatch(batchsize=1, ix=None):
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
            
#     return embeddings, torch.tensor(targets), minibatch_ix
    return embeddings, torch.tensor(targets)

In [189]:
e,t,i = get_minibatch(batchsize=num_samples)

In [190]:
len(e)

792

In [191]:
len(t)

792

In [265]:
len(e)

792

In [266]:
max([len(emb) for emb in e])

546

In [199]:
# all words same len of emb size 371 
set([len(emb[i]) for emb in e for i in range(len(emb))])

{371}

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Seq2Seq -- with attention

2 layer rnn..

In [22]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

371

In [163]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
# with EOS pos
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [30]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [53]:
VOCAB_SIZE = len(vocab_emb)

In [101]:
len(dict)

71

In [102]:
# pos_dict['EOS'] = len(pos_dict)

In [104]:
# EOS tag
# vocab_emb['EOS'] = torch.rand([300],dtype=torch.float64)

In [256]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        # what is the input size fo self.embedding...? and is self.embedding necessary
#         self.embedding = nn.Embedding(input_size, emb_size)
        
        
        
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.gru = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size,num_layers=2)
        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
    
        self.fc = nn.Linear(self.hidden_size*2,1).double()

    def forward(self, input, hidden):
        # confirm dims here...
#         embedded = self.embedding(input).view(1, 1, -1)
        
        embedded = torch.cat(input).view(len(input), 1, -1).double()
        
#         output = embedded
        
#         output, hidden = self.gru(self.emb_size, self.hidden_size)
#         output, hidden = self.gru(output)
        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)
        print(gru_out.size(),_.size())
    
#         lstm_out, _ = self.lstm(embedding.view(len(sentence), -1))
        out = self.fc(hid_out)
        
        # any further processing -- int? softmax...? 
        return out
        return output, hidden

    def initHidden(self):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, 1, self.hidden_size, device=device).double()


In [257]:
x,y = get_minibatch(batchsize=1)

In [258]:
torch.cat(x[0]).view(len(x[0]),1, -1).shape

torch.Size([53, 1, 371])

In [251]:
# need to be able to pass any/all into net
# encoder = EncoderRNN(len(x[0])).to(device)

In [259]:
encoder = EncoderRNN()

In [260]:
_hidden = encoder.initHidden()

In [216]:
# encoder = encoder.float()

In [261]:
out = encoder(x[0],_hidden)

torch.Size([53, 1, 128]) torch.Size([2, 1, 128])


In [255]:
torch.cat((test_hid[-2,:,:], test_hid[-1,:,:]), dim = 1).size()

torch.Size([1, 256])

In [262]:
len(x[0])

53

In [263]:
out.shape

torch.Size([1, 1])

In [264]:
out

tensor([[0.1462]], dtype=torch.float64, grad_fn=<AddmmBackward>)

In [315]:
class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
    def __init__(self, hidden_size=HIDDEN_SIZE, output_size=1, dropout_p=0.1, max_length=MAX_LENGTH):    
    
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        
        #fc
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = torch.cat(input).view(len(input), 1, -1)
        embedded = input
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        # unnecessary...predicting one value
#         output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [316]:
decoder = AttnDecoderRNN()

In [273]:
decoder_hidden = decoder.initHidden()

In [280]:
_hidden.shape

torch.Size([1, 1, 128])